# Number of above zero degree C days

In [1]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')

import get_transect_data

sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsO/"

In [3]:
#Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'], ['CESM_C30', 'CESM_C31']]
#Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'], ['CESM_C30','CESM_C31','CESM_C32','CESM_C33'] ]# modified rain model
#Sim_name = ['S1','S2','S3']

#Dec 15, 2020: ProjectionsO
Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%s'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves
Sim_name = ['S1','S2','S3']

In [4]:
def getData(i,var_):
    Data = []
    Time = []
    for s in Simulations[i]:
        #print (i, s)
        infile = os.path.join(path,s)
        
        Tim1, Dat1 = get_transect_data.get2DMatrix(infile,varnames=[var_])
        a1 = len(Dat1[0,:,0,0])%73
        a2 = len(Dat1[0,:,0,0]) - a1
        
        Tim = Tim1[:a2]
        Dat = Dat1[:,:a2,:,:]
        Time.append(Tim)
        Data.append(Dat)
        #print ('HERE',a1,a2, Tim1.shape)
    d = dict()
    key_v = Sim_name[i] + '_' + var_
    key_t = Sim_name[i] + '_' + 'time'
    d[key_v] = Data
    d[key_t] = Time
    
    return d

In [5]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data(var):
    global DataS1, DataS2, DataS3
    prod_x=partial(getData, var_=var)
    DataS1, DataS2, DataS3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataS1, DataS2,DataS3]
    return Data

3 709 45 85
3 1515 45 85
3 2182 45 85
3 4774 45 85
3 3499 45 85
3 4700 45 85
3 1375 45 85
3 803 45 85
3 1387 45 85
3 219 45 85


In [6]:
def getData_All(var_loc):
    DataSimOrig = dict()
    #DataWC = call_data(var=var_loc)
    DataTemp = call_data(var='temperature')
    #DataVol = call_data(var='cell_volume')
    #print (DataTemp[0].keys())
    for i in range(len(Sim_name)): #number of simulations
        data_wc = []
        L = len(Simulations[i])
        Time = []
        DDays  = []
        DegDays = []
        for k in range(L): #k is for the number of directories for a simulation
            #key = list(DataWC[i].keys())[0] #[0] ='water_content', [1] = 'time'
            key_temp = list(DataTemp[i].keys())[0]
            key_time = list(DataTemp[i].keys())[1]
            
            data_temp = np.array(DataTemp[i][key_temp][k])
            data_time = np.array(DataTemp[i][key_time][k])
            #print (data_wc.shape,data_temp.shape,data_time.shape)
            nvar, cycles, xnum, znum = data_temp.shape
            
            for cyc in range(cycles):
                temp_index = np.where(data_temp[nvar-1,cyc,:,:] > 273.15)
                temp1 = data_temp[nvar-1,cyc,0,82]
                temp_index1 = np.where(data_temp[nvar-1,cyc,:,:] <= 273.15)
                #print (temp1)
                if len(temp_index[0]) > 0:
                #if temp1 > 273.15:
                    DDays.append(5)
                    sum1 = np.mean(data_temp[nvar-1,cyc,:,:][temp_index])
                    #sum1 = temp1
                    
                    DegDays.append(sum1 - 273.15)
                    #print (sum1-273.15)
                    #break
                else:
                    DDays.append(0)
                    sum1 = np.mean(data_temp[nvar-1,cyc,:,:][temp_index1])
                    #print (temp_index1, data_temp[nvar-1,cyc,:,:][temp_index1])
                    DegDays.append(sum1 - 273.15)

                
                Time.append(data_time[cyc])
                
                    
        DataSimOrig[Sim_name[i] + 'day'] = np.array(DDays)
        DataSimOrig[Sim_name[i] + '_time'] = np.array(Time)
        DataSimOrig[Sim_name[i] + 'degday'] = np.array(DegDays)
    return DataSimOrig

In [7]:
DataSim = getData_All(var_loc='water_content')

In [8]:
def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
                
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [ ]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/Nov20/'
def Plot_PermafrostThawRate():
    fig, axs = plt.subplots(1,1, figsize=(5,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    #Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']
    Labels = ['no-flow', 'free-flow', 'hybrid-flow']
    Colors = ['k','b','r','r']
    T = ['a','b','c','d']
    Keys = list(DataSim.keys())
    print (Keys)
    
    for i in range(3):
        #print (DataSim[Keys[2*i]])
        d = DataSim[Keys[3*i]]
        #print (d)
        #break
        d = np.reshape(d,(-1,73)).sum(axis=1)
        
        d_avg5yr = get_moving_average(d,m=5)
        
        t = np.reshape(DataSim[Keys[3*i+1]],(-1,73)).mean(axis=1)

        axs.plot(2006 + t, d_avg5yr,color=Colors[i],label=Labels[i])
        #axs[i].text(2006,3.7,'(%s)'%T[i],fontsize=12, fontweight='normal')
        axs.set_xlim((2007, 2100))
        #axs.set_ylim((0, 7))
        #axs.set_yticks(np.arange(0,20.1,10))
        axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('Thawing degree-days',fontsize=12, fontweight='normal')
        axs.legend(loc='upper left',labelspacing=.1)
            
        #axs.set_ylim(135,138)
    print (outfile)
    #plt.savefig(outfile+'above_freezing_days.pdf', bbox_inches='tight', dpi=500)
    plt.show()



In [ ]:
Plot_PermafrostThawRate()

In [13]:
%matplotlib qt
outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/Nov20/'
def Plot_DegDays():
    fig, axs = plt.subplots(1,1, figsize=(5,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    #Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']
    Labels = ['no-flow', 'free-flow', 'hybrid-flow']
    Colors = ['k','b','r','r']
    T = ['a','b','c','d']
    Keys = list(DataSim.keys())
    print (Keys)
    
    for i in range(3):
        #print (DataSim[Keys[2*i]])
        d = DataSim[Keys[3*i+2]]
        #print (d)
        #break
        
        d = np.reshape(d,(-1,73))
        #print (d.shape)
        DD = []
        for kk, ix in enumerate(d):
            
            d1 = np.where(ix>0)[0]
            
            if len(d1)>0:
                #print (d1,ix[d1])
                
                DD.append(np.sum(ix[d1])*5)#*len(ix[d1]))
                #print (kk, len(ix),len(d1),np.sum(ix[d1]))
            #break
            #print (DD)
        d_avg5yr = get_moving_average(DD,m=5)
        
        t = np.reshape(DataSim[Keys[3*i+1]],(-1,73)).mean(axis=1)

        axs.plot(2006 + t, d_avg5yr,color=Colors[i],label=Labels[i])
        #axs.plot(d_avg5yr,color=Colors[i],label=Labels[i])
        
        
        axs.set_xlim((2007, 2100))
        axs.set_ylim((200, 1050))
        axs.set_yticks(np.arange(200,1050.,200))
        axs.set_xlabel('Calendar year',fontsize=12, fontweight='normal')
        
        axs.set_ylabel('TDD [\u00B0C d]',fontsize=12, fontweight='normal')
        axs.legend(loc='upper left',labelspacing=.1)
            
        #axs.set_ylim(135,138)
    print (outfile)
    #plt.savefig(outfile+'/TDD.pdf', bbox_inches='tight', dpi=500)
    plt.show()




In [14]:
Plot_DegDays()

['S1day', 'S1_time', 'S1degday', 'S2day', 'S2_time', 'S2degday', 'S3day', 'S3_time', 'S3degday']
/Users/ajc/Documents/MyPapers/TroughBC/figures/Nov20/


In [ ]:
44*35